### Whisper 快速使用指南
Whisper 是由 OpenAI 提供的一種開源神經網路模型，用於自動語音辨識（Automatic Speech Recognition, ASR）。它能夠執行語言識別（language identification）和語音翻譯（speech translation），支持多語種語音轉錄，甚至可以處理音頻品質較差或有大量背景雜訊的情況。

以下步驟將說明如何使用此程式將影音轉換為文字：
1. 在「執行階段」選單中，選擇「變更執行階段類型」，然後在彈出的視窗中選擇 `T4 GPU`。
2. 每次打開此文件時，點擊「執行階段」中的「全部執行」或按下 `Ctrl + F9`。
3. 當程序執行至第 4 段時，請選擇要提取文字的資料類型（YouTube 連結 / MP3 / MP4），然後點擊「確認」按鈕。
4. 當程序執行至第 5 段時，請輸入資料位置（網址或檔案路徑），然後按下 `Enter` 鍵。
5. 在右側的資料夾中找到並下載已轉換的 TXT 檔案。

In [ ]:
"""
1. 下載必要套件，並引用它們
"""
!pip install yt-dlp                     # 用來下載Youtube影片
!pip install git+https://github.com/openai/whisper.git   # 用來下載 Whisper
!sudo apt update && sudo apt install ffmpeg      # 用 Linux 指令下載 ffmpeg，用來將 "影片檔" 轉為 "錄音檔"
!pip install librosa                    # 用來提取聲音的特徵

# 引用套件
import whisper
import time
import librosa
import re
import yt_dlp as youtube_dl
import subprocess
from google.colab import widgets
import ipywidgets as widgets
from IPython.display import display
import os

In [ ]:
"""
2. 選擇想使用的聲音辨識模型，基本上 base 就夠了

  若確定音檔內容皆為英文，則可於模型名稱後加上 ".en"

  若不確定音檔所使用的語言或音檔內容不是純英文，則得移除模型名稱後的 ".en"
"""
# model = whisper.load_model("tiny.en")
model = whisper.load_model("base")
# model = whisper.load_model("small.en")
# model = whisper.load_model("medium.en")
# model = whisper.load_model("large")

In [3]:
"""
3. 這裡是處理各資料類型的函式
"""
def youtube_filepath():
    url = input("Enter a YouTube video URL: ")

    # Create a youtube-dl options dictionary
    ydl_opts = {
        # Specify the format as bestaudio/best
        'format': 'bestaudio/best',
        # Specify the post-processor as ffmpeg to extract audio and convert to mp3
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        # Specify the output filename as the video title
        'outtmpl': '%(title)s.%(ext)s',
    }

    # Download the video and extract the audio
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

    # Get the path of the file
    file_path = ydl.prepare_filename(ydl.extract_info(url, download=False))
    file_path = file_path.replace('.webm', '.mp3')
    file_path = file_path.replace('.m4a', '.mp3')
    return file_path


def mp4_filepath():
  file_path = input("Enter the MP4 filepath: ")
  command = "ffmpeg -i '{}' -vn -ar 44100 -ac 2 -b:a 192k '{}'".format(file_path.replace("/content/", ""), file_path.replace(".mp4", ".mp3"))
  !$command
  return file_path.replace(".mp4", ".mp3")


In [ ]:
"""
4. 執行本段程式, 選擇欲轉換的資料類型, 並按下確認按鈕
"""
file_type = None

# Create a dropdown widget for file type selection
file_type_dropdown = widgets.Dropdown(
    options=['YouTube Link', 'MP4', 'MP3'],
    description='資料類型: ',
)

# Create a button to trigger the processing
process_button = widgets.Button(description="確定")

# Output widget to display results
output = widgets.Output()

# Define a function to handle the processing
def process_file_type(b):
    with output:
        output.clear_output()  # Clear previous outputs
        print(f"選擇的資料類型為 {file_type_dropdown.value}")
        global file_type
        file_type = file_type_dropdown.value


# Link the button to the processing function
process_button.on_click(process_file_type)

# Display the dropdown, button, and output widget
display(file_type_dropdown, process_button, output)


In [ ]:
"""
5. 輸入網址或資料連結後, 按下 Enter 以開始引用第 3 段的函式
"""
if file_type == 'YouTube Link':
    print("User selected YouTube Link.")
    file_path = youtube_filepath()
    print(file_path)

elif file_type == 'MP4':
    print("User selected MP4.")
    file_path = mp4_filepath()
    print(file_path)

elif file_type == 'MP3':
    print("User selected MP3.")
    file_path = input("Enter the MP3 filepath: ")
    print(file_path)

else:
    print("User selected YouTube Link.")
    file_path = youtube_filepath()
    print(file_path)

In [ ]:
"""
6. 此階段的程式會用 Whisper 模型開始將影音轉為文字
  txt 檔案將會被儲存於右側資料夾之中
  可以直接下載
"""

# Get the duration
duration = librosa.get_duration(path=file_path)
start = time.time()
result = model.transcribe(file_path)
end = time.time()
seconds = end - start

result

print("Video length:", duration, "seconds")
print("Transcription time:", seconds)

# Split result["text"]  on !,? and . , but save the punctuation
sentences = re.split("([!?.])", result["text"])

# Join the punctuation back to the sentences
sentences = ["".join(i) for i in zip(sentences[0::2], sentences[1::2])]
text = "\n\n".join(sentences)
for s in sentences:
  print(s)

# Save the file as .txt
name = "".join(file_path) + ".txt"
with open(name, "w") as f:
  f.write(text)

print("\n\n", "-"*100, "\n\nYour transcript is here:", name)